# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

В данном задании необходимо выбрать оптимальную модель для предсказания стоимости подержанных автомобилей исходя из набора признаков: модели, года выпуска, пробега и т.д. Критериями оптимальности является точность предсказания (RMSE менее 2500) и время обучения и предсказания. Задание будет разбито на несколько этапов: импорт и предобработка данных, сравнение моделей и выводы

### Импорт данных и библиотек

In [481]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import numpy as np
import lightgbm as lgb
from sklearn.datasets import load_boston
from pandas import DataFrame
import matplotlib.pyplot as plt
import catboost as cb
from sklearn.linear_model import LinearRegression

Импорт библиотек

In [482]:
data = pd.read_csv('/datasets/autos.csv')
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [483]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [484]:
data.describe(include='all')

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
count,354369,354369.000000,316879,354369.000000,334536,354369.000000,334664,354369.000000,354369.000000,321474,354369,283215,354369,354369.0,354369.000000,354369
unique,271174,NaN,8,NaN,2,NaN,250,NaN,NaN,7,40,2,109,NaN,NaN,179150
top,2016-03-24 14:49:47,NaN,sedan,NaN,manual,NaN,golf,NaN,NaN,petrol,volkswagen,no,2016-04-03 00:00:00,NaN,NaN,2016-04-06 13:45:54
freq,7,NaN,91457,NaN,268251,NaN,29232,NaN,NaN,216352,77013,247161,13719,NaN,NaN,17
mean,NaN,4416.656776,NaN,2004.234448,NaN,110.094337,NaN,128211.172535,5.714645,NaN,NaN,NaN,NaN,0.0,50508.689087,NaN
std,NaN,4514.158514,NaN,90.227958,NaN,189.850405,NaN,37905.341530,3.726421,NaN,NaN,NaN,NaN,0.0,25783.096248,NaN
min,NaN,0.000000,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,1050.000000,NaN,1999.000000,NaN,69.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30165.000000,NaN
50%,NaN,2700.000000,NaN,2003.000000,NaN,105.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,49413.000000,NaN
75%,NaN,6400.000000,NaN,2008.000000,NaN,143.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,71083.000000,NaN


Загрузка и первичное ознакомление с данными

### Предобработка данных

Первичный анализ позволяет сделать вывод о том что таблица содержит пропуски, дублирующие и аномальные значения и неинформативные колонки. Первым шагом удалим дубликаты

In [485]:
data = data.drop_duplicates().reset_index()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354365 entries, 0 to 354364
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   index              354365 non-null  int64 
 1   DateCrawled        354365 non-null  object
 2   Price              354365 non-null  int64 
 3   VehicleType        316875 non-null  object
 4   RegistrationYear   354365 non-null  int64 
 5   Gearbox            334532 non-null  object
 6   Power              354365 non-null  int64 
 7   Model              334660 non-null  object
 8   Kilometer          354365 non-null  int64 
 9   RegistrationMonth  354365 non-null  int64 
 10  FuelType           321470 non-null  object
 11  Brand              354365 non-null  object
 12  NotRepaired        283211 non-null  object
 13  DateCreated        354365 non-null  object
 14  NumberOfPictures   354365 non-null  int64 
 15  PostalCode         354365 non-null  int64 
 16  LastSeen           3

Удалим неинформативные колонки. В колонке NumberOfPictures все значения нулевые, соответственно, ее тоже можно удалить

In [486]:
data = data.drop(['index', 'DateCrawled', 'RegistrationMonth', 'DateCreated', 'PostalCode', 'LastSeen', 'NumberOfPictures'], axis = 1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354365 entries, 0 to 354364
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             354365 non-null  int64 
 1   VehicleType       316875 non-null  object
 2   RegistrationYear  354365 non-null  int64 
 3   Gearbox           334532 non-null  object
 4   Power             354365 non-null  int64 
 5   Model             334660 non-null  object
 6   Kilometer         354365 non-null  int64 
 7   FuelType          321470 non-null  object
 8   Brand             354365 non-null  object
 9   NotRepaired       283211 non-null  object
dtypes: int64(4), object(6)
memory usage: 27.0+ MB


Часть значений в колонках пропущена. Оценим какая часть выборки будет утрачена при удалении строк хотя бы с одним пропущенным значением

In [487]:
data_drop = data.dropna()
data_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 245810 entries, 3 to 354363
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             245810 non-null  int64 
 1   VehicleType       245810 non-null  object
 2   RegistrationYear  245810 non-null  int64 
 3   Gearbox           245810 non-null  object
 4   Power             245810 non-null  int64 
 5   Model             245810 non-null  object
 6   Kilometer         245810 non-null  int64 
 7   FuelType          245810 non-null  object
 8   Brand             245810 non-null  object
 9   NotRepaired       245810 non-null  object
dtypes: int64(4), object(6)
memory usage: 20.6+ MB


Удаление всех строк с пропущенными значениями приводит к потере ~30% строк. Для выборки в несколько сот тысяч записей потеря несущественна. При этом отсутствует надежный способ заполнить отсутствующие значения. Оценим распределение данных в получившейся выборке

Удаление всех строк с пропущенными значениями приводит к потере ~30% строк. Заполним пропущенные значения. Принцип заполнения: в тех случаях когда один из возможных вариантов очевидно хуже других заполним пропущенные значения этим наихудшим вариантом исходя из того соображения что продавец должен быть заинтересован представить свой автомобиль в максимально выгодном свете. В случаях когда очевидно наихудший вариант отсутствует подставим значение unknown. Пройдемся по колонкам где есть пропущенные значения

In [488]:
data['VehicleType'].value_counts()

sedan          91457
small          79830
wagon          65165
bus            28775
convertible    20203
coupe          16161
suv            11996
other           3288
Name: VehicleType, dtype: int64

VehicleType - отсутствует очевидно худший вариант. Заполним пропущенные значения unknown

In [489]:
data['VehicleType'] = data['VehicleType'].fillna('unknown')
data['VehicleType'].value_counts()

sedan          91457
small          79830
wagon          65165
unknown        37490
bus            28775
convertible    20203
coupe          16161
suv            11996
other           3288
Name: VehicleType, dtype: int64

In [490]:
data['Gearbox'].value_counts()

manual    268249
auto       66283
Name: Gearbox, dtype: int64

Автоматические КПП котируются выше поэтому будем считать что пропущенные значения соответствуют ручным КПП

In [491]:
data['Gearbox'] = data['Gearbox'].fillna('manual')
data['Gearbox'].value_counts()

manual    288082
auto       66283
Name: Gearbox, dtype: int64

Model - заполняем unknown

In [492]:
data['Model'] = data['Model'].fillna('unknown')
data['Model'].value_counts()

golf                  29232
other                 24420
3er                   19761
unknown               19705
polo                  13066
                      ...  
i3                        8
rangerover                4
serie_3                   4
range_rover_evoque        2
serie_1                   2
Name: Model, Length: 251, dtype: int64

In [493]:
data['FuelType'].value_counts()

petrol      216349
gasoline     98719
lpg           5310
cng            565
hybrid         233
other          204
electric        90
Name: FuelType, dtype: int64

FuelType - пропуски заполняем unknown. Petrol и gasoline - синонимы, означающие бензин. Их можно объединить

In [494]:
data['FuelType'] = data['FuelType'].fillna('unknown')
data['FuelType'] = data['FuelType'].replace('gasoline', 'petrol')
data['FuelType'].value_counts()

petrol      315068
unknown      32895
lpg           5310
cng            565
hybrid         233
other          204
electric        90
Name: FuelType, dtype: int64

In [495]:
data['NotRepaired'].value_counts()

no     247158
yes     36053
Name: NotRepaired, dtype: int64

Отсутствие ремонта является плюсом, соответственно, пропущенные значения с большой вероятностью говорят о том что ремонт имет место

In [496]:
data['NotRepaired'] = data['NotRepaired'].fillna('no')
data['NotRepaired'].value_counts()

no     318312
yes     36053
Name: NotRepaired, dtype: int64

In [497]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354365 entries, 0 to 354364
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             354365 non-null  int64 
 1   VehicleType       354365 non-null  object
 2   RegistrationYear  354365 non-null  int64 
 3   Gearbox           354365 non-null  object
 4   Power             354365 non-null  int64 
 5   Model             354365 non-null  object
 6   Kilometer         354365 non-null  int64 
 7   FuelType          354365 non-null  object
 8   Brand             354365 non-null  object
 9   NotRepaired       354365 non-null  object
dtypes: int64(4), object(6)
memory usage: 27.0+ MB


Пропуски заполнены наиболее подходящими значениями. Переходим к поиску аномалий в количественных признаках

In [498]:
data.describe()

,Price,RegistrationYear,Power,Kilometer
count,354365.000000,354365.000000,354365.000000,354365.000000
mean,4416.679830,2004.234481,110.093816,128211.363989
std,4514.176349,90.228466,189.851330,37905.083858
min,0.000000,1000.000000,0.000000,5000.000000
25%,1050.000000,1999.000000,69.000000,125000.000000
50%,2700.000000,2003.000000,105.000000,150000.000000
75%,6400.000000,2008.000000,143.000000,150000.000000
max,20000.000000,9999.000000,20000.000000,150000.000000


Обращают на себя внимание следующие факторы. 
1. Цена не может быть нулевой или близкой к нолю. Не существуют однозначного определения невозможно низкой цены, но цены менее 10% от медианы можно считать маловероятными. Их предлагается удалить

In [499]:
data = data[data['Price'] > 350]
data.describe()

,Price,RegistrationYear,Power,Kilometer
count,325510.000000,325510.000000,325510.000000,325510.000000
mean,4796.349485,2004.066591,113.869052,128225.676631
std,4518.003885,66.339124,187.697745,37174.206264
min,355.000000,1000.000000,0.000000,5000.000000
25%,1400.000000,1999.000000,75.000000,125000.000000
50%,3000.000000,2003.000000,105.000000,150000.000000
75%,6899.000000,2008.000000,143.000000,150000.000000
max,20000.000000,9999.000000,20000.000000,150000.000000


2. Наиболее поздним годом регистрации является 2018. Машины старше 30 лет можно удалить так как они с большой вероятностью непригодны к использованию по основному назначению и должны оцениваться по критериям отличным от машин в рабочем состоянии

In [500]:
data = data[data['RegistrationYear'] > 1986]
data = data[data['RegistrationYear'] < 2017]
data.describe()

,Price,RegistrationYear,Power,Kilometer
count,306368.000000,306368.000000,306368.000000,306368.000000
mean,4825.583217,2003.311204,115.547747,128609.515354
std,4532.363607,5.633882,184.763748,36664.836927
min,355.000000,1987.000000,0.000000,5000.000000
25%,1400.000000,1999.000000,75.000000,125000.000000
50%,3100.000000,2003.000000,109.000000,150000.000000
75%,6900.000000,2007.000000,145.000000,150000.000000
max,20000.000000,2016.000000,20000.000000,150000.000000


3. Мощность предлагаемых автомобилей можно ограничить диапазоном 50-300 л.с.

In [501]:
data = data[data['Power'] >= 50]
data = data[data['Power'] <= 300]
data.describe()

,Price,RegistrationYear,Power,Kilometer
count,273813.000000,273813.000000,273813.000000,273813.000000
mean,4991.555543,2003.460245,121.303623,128312.479685
std,4543.879753,5.554089,47.960411,36471.429273
min,355.000000,1987.000000,50.000000,5000.000000
25%,1500.000000,1999.000000,82.000000,125000.000000
50%,3400.000000,2003.000000,116.000000,150000.000000
75%,7000.000000,2007.000000,150.000000,150000.000000
max,20000.000000,2016.000000,300.000000,150000.000000


4. Проанализируем колонку Kilometer где значения выглядят "круглыми"

In [502]:
data['Kilometer'].value_counts()

150000    180939
125000     29786
100000     11859
90000       9680
80000       8481
70000       7395
60000       6496
50000       5377
40000       4278
30000       3628
20000       2952
5000        2147
10000        795
Name: Kilometer, dtype: int64

Данные о пробеге указываются с округлением 5 000, 10 000 или 25 000 километров в зависимости от его абсолютного значения. Максимально допустимое значение составляет 150 тысяч километров. По всей видимости это связано с тем что пользователю предлагается сделать выбор из нескольких округленных значений, а не указывать фактическую величину

Добавим колонку с возрастом автомобиля. Этот признак меняется сильнее чем год регистрации что улучшит обучение. При тестировании моделей она заменит колонку RegistrationYear

In [503]:
data['Age'] = 2017 - data['RegistrationYear']
data.describe()

,Price,RegistrationYear,Power,Kilometer,Age
count,273813.000000,273813.000000,273813.000000,273813.000000,273813.000000
mean,4991.555543,2003.460245,121.303623,128312.479685,13.539755
std,4543.879753,5.554089,47.960411,36471.429273,5.554089
min,355.000000,1987.000000,50.000000,5000.000000,1.000000
25%,1500.000000,1999.000000,82.000000,125000.000000,10.000000
50%,3400.000000,2003.000000,116.000000,150000.000000,14.000000
75%,7000.000000,2007.000000,150.000000,150000.000000,18.000000
max,20000.000000,2016.000000,300.000000,150000.000000,30.000000


In [504]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,Age
count,273813.000000,273813.000000,273813.000000,273813.000000,273813.000000
mean,4991.555543,2003.460245,121.303623,128312.479685,13.539755
std,4543.879753,5.554089,47.960411,36471.429273,5.554089
min,355.000000,1987.000000,50.000000,5000.000000,1.000000
25%,1500.000000,1999.000000,82.000000,125000.000000,10.000000
50%,3400.000000,2003.000000,116.000000,150000.000000,14.000000
75%,7000.000000,2007.000000,150.000000,150000.000000,18.000000
max,20000.000000,2016.000000,300.000000,150000.000000,30.000000


Данные подготовлены для построения моделей

## Обучение моделей

### Дерево решений

In [505]:
features_tree = data.drop(['RegistrationYear', 'Price', 'VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired'], axis = 1)
target_tree = data['Price']

Выделены признаки с удалением категориальных и целевой признак

In [506]:
features_train, features_aux, target_train, target_aux = train_test_split(features_tree, target_tree, test_size=0.4, random_state=12345)
features_train.shape

(164287, 3)

In [507]:
features_valid, features_test, target_valid, target_test = train_test_split(features_aux, target_aux, test_size=0.5, random_state=12345)
features_valid.shape

(54763, 3)

Выборка разделена на тестовую, валидационную и обучающую в рекомендованном соотношении 3:1:1

Измерим отдельно длительности обучения и длительность предсказания. Длительность предсказания рассчитывается как общая длительность минус длительность обучения

In [508]:
%%time
tree_model = None
tree_result = 0
for depth in range(1,5):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)

CPU times: user 297 ms, sys: 0 ns, total: 297 ms
Wall time: 302 ms


In [509]:
%%time
tree_model = None
tree_result = 0
for depth in range(1,5):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    result = mean_squared_error(target_valid, predictions) ** .5
    if result > tree_result:
        tree_model = model
        tree_result = result
        tree_depth = depth
print('RMSE лучшей модели:', tree_result )
print('Оптимальная глубина дерева:', tree_depth)

RMSE лучшей модели: 4852.07828104798
Оптимальная глубина дерева: 1
CPU times: user 1.49 s, sys: 1.25 s, total: 2.74 s
Wall time: 2.75 s


Вывод: дерево решений обучается на обучающей выборке за 1 секунду и делает предсказание на тестовой выборке за 3 секунды. RMSE в заданных гиперпараметрах 4852. Оптимальное значение гиперпараметра Глубина дерева - 1

### LightGBM

Определим параметры

In [510]:
params = {'task': 'train', 'boosting': 'gbdt', 'objective': 'regression', 'num_leaves': 10, 'learning_rate': 0.001, 'feature_fraction': 0.95, 'metric': {'l2','l1'}, 'verbose': -1}

Загрузка данных и обучение модели

In [511]:
%%time
lgb_train = lgb.Dataset(features_train, target_train)
lgb_eval = lgb.Dataset(features_valid, target_valid, reference=lgb_train)
model = lgb.train(params, train_set=lgb_train, valid_sets=lgb_eval)

[1]	valid_0's l2: 2.08783e+07	valid_0's l1: 3611.19
[2]	valid_0's l2: 2.08507e+07	valid_0's l1: 3608.71
[3]	valid_0's l2: 2.08232e+07	valid_0's l1: 3606.24
[4]	valid_0's l2: 2.07957e+07	valid_0's l1: 3603.78
[5]	valid_0's l2: 2.07683e+07	valid_0's l1: 3601.31
[6]	valid_0's l2: 2.07409e+07	valid_0's l1: 3598.85
[7]	valid_0's l2: 2.07136e+07	valid_0's l1: 3596.4
[8]	valid_0's l2: 2.06864e+07	valid_0's l1: 3593.95
[9]	valid_0's l2: 2.06592e+07	valid_0's l1: 3591.51
[10]	valid_0's l2: 2.0632e+07	valid_0's l1: 3589.06
[11]	valid_0's l2: 2.06049e+07	valid_0's l1: 3586.62
[12]	valid_0's l2: 2.05779e+07	valid_0's l1: 3584.18
[13]	valid_0's l2: 2.05509e+07	valid_0's l1: 3581.75
[14]	valid_0's l2: 2.0524e+07	valid_0's l1: 3579.32
[15]	valid_0's l2: 2.04971e+07	valid_0's l1: 3576.89
[16]	valid_0's l2: 2.04703e+07	valid_0's l1: 3574.46
[17]	valid_0's l2: 2.04435e+07	valid_0's l1: 3572.04
[18]	valid_0's l2: 2.04168e+07	valid_0's l1: 3569.62
[19]	valid_0's l2: 2.03901e+07	valid_0's l1: 3567.2
[20]	v

Обучение занимает 2 минуты. Предсказание 

In [512]:
%%time
predictions = model.predict(features_valid)
rmse = mean_squared_error(target_test, predictions) ** .5
print(rmse)

4544.269168773121
CPU times: user 224 ms, sys: 2.97 ms, total: 227 ms
Wall time: 292 ms


Вывод: LightGBM требует 2 минуты для обучения и 1 секунду предсказания. RMSE модели в заданных гиперпараметрах 4544. Попробуем другие модели 

### Линейная регрессия

In [513]:
%%time
model = LinearRegression()
model.fit(features_train, target_train)

CPU times: user 9.7 ms, sys: 3 ms, total: 12.7 ms
Wall time: 11.4 ms


LinearRegression()

In [514]:
%%time
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
result = mean_squared_error(target_valid, predictions) ** .5
print(result)

2825.2161120032965
CPU times: user 19.2 ms, sys: 393 µs, total: 19.6 ms
Wall time: 14.9 ms


Вывод: линейная регрессия обучается и делает предсказания менее чем за секунду. RMSE составляет 2825. Попробуем другие модели

### CatBoost

In [532]:
train_dataset = cb.Pool(features_train, target_train) 
test_dataset = cb.Pool(features_valid, target_valid)
model = cb.CatBoostRegressor(loss_function='RMSE')

In [533]:
%%time
grid = {'iterations': [100, 150, 200], 'learning_rate': [0.03, 0.1], 'depth': [2, 4, 6, 8], 'l2_leaf_reg': [0.2, 0.5, 1, 3]}
model.grid_search(grid, train_dataset)

0:	learn: 6576.1095399	test: 6602.7588372	best: 6602.7588372 (0)	total: 15.3ms	remaining: 1.52s
1:	learn: 6423.5846230	test: 6450.2797080	best: 6450.2797080 (1)	total: 29.4ms	remaining: 1.44s
2:	learn: 6276.6260444	test: 6303.4005259	best: 6303.4005259 (2)	total: 42.6ms	remaining: 1.38s
3:	learn: 6134.9911455	test: 6161.3977465	best: 6161.3977465 (3)	total: 56.2ms	remaining: 1.35s
4:	learn: 5997.9656398	test: 6024.1467882	best: 6024.1467882 (4)	total: 68.2ms	remaining: 1.29s
5:	learn: 5866.3646941	test: 5892.4168626	best: 5892.4168626 (5)	total: 79.7ms	remaining: 1.25s
6:	learn: 5739.1651956	test: 5765.5593263	best: 5765.5593263 (6)	total: 91.4ms	remaining: 1.21s
7:	learn: 5616.9271735	test: 5643.3950019	best: 5643.3950019 (7)	total: 106ms	remaining: 1.22s
8:	learn: 5499.8535161	test: 5526.6693614	best: 5526.6693614 (8)	total: 120ms	remaining: 1.21s
9:	learn: 5385.9380721	test: 5412.8583289	best: 5412.8583289 (9)	total: 133ms	remaining: 1.19s
10:	learn: 5275.4277063	test: 5302.5002992	

{'params': {'depth': 8,
  'iterations': 200,
  'learning_rate': 0.1,
  'l2_leaf_reg': 0.5},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               4

In [534]:
%%time
predictions = model.predict(features_valid)
rmse = (np.sqrt(mean_squared_error(target_valid, predictions)))
print(rmse)

2002.9617353276653
CPU times: user 17.9 ms, sys: 100 µs, total: 18 ms
Wall time: 15.9 ms


Обучение модели занимает 4 минуты, предсказание 1 секунда. RMSE валидационной выборки в заданных гиперпараметрах 2002

## Анализ моделей

Для решения поставленной задачи проведено сравнение четырех моделей. Двух прямых: дерево решений, линейная регрессия и двух градиентных: LightGBM и CatBoost. Для экспресс-оценки наилучшим образом подходит линейная регрессия. За несколько секунд он способна дать предсказание всего на 7% выше целевого значения. Ее можно использовать, например, для подсвечивания диапазона стоимости автомобиля при внесении пользователем его параметров. Наибольшую точность, существенно превосходящую требуемую по условиям задачи дает CatBoost. Ее можно использовать для точного расчета стоимости при прямом запросе пользователя, по нажатию условной кнопки "Рассчитать стоимость"

Обобщим полученные результаты

In [536]:
headings = ['Модель', 'Время обучения, секунды', 'Время предсказания, секунды', 'RMSE']
figures = [['Дерево решений', 1, 3, 4852], ['LightGBM', 120, 1, 4544], ['Лин. регрессия', 1, 1, 2825], ['CatBoost', 240, 1, 2002]]
summary = pd.DataFrame(data=figures, columns=headings)
print(summary)

           Модель  Время обучения, секунды  Время предсказания, секунды  RMSE
0  Дерево решений                        1                            3  4852
1        LightGBM                      120                            1  4544
2  Лин. регрессия                        1                            1  2825
3        CatBoost                      240                            1  2002


Вывод: исходя из результатов тестирования единственной моделью отвечающей условиям задачи является CatBoost. Хорошие результаты показывает линейная регрессия. При высокой скорости обучения результат всего на 10-15% хуже заданного

# Тестирование лучшей модели

In [540]:
%%time
train_dataset = cb.Pool(features_train, target_train) 
test_dataset = cb.Pool(features_test, target_test)
model = cb.CatBoostRegressor(loss_function='RMSE')
grid = {'iterations': [100, 150, 200], 'learning_rate': [0.03, 0.1], 'depth': [2, 4, 6, 8], 'l2_leaf_reg': [0.2, 0.5, 1, 3]}
model.grid_search(grid, train_dataset)
predictions = model.predict(features_test)
rmse = (np.sqrt(mean_squared_error(target_test, predictions)))
print(rmse)

0:	learn: 6576.1095399	test: 6602.7588372	best: 6602.7588372 (0)	total: 12.1ms	remaining: 1.2s
1:	learn: 6423.5846230	test: 6450.2797080	best: 6450.2797080 (1)	total: 23.7ms	remaining: 1.16s
2:	learn: 6276.6260444	test: 6303.4005259	best: 6303.4005259 (2)	total: 35.5ms	remaining: 1.15s
3:	learn: 6134.9911455	test: 6161.3977465	best: 6161.3977465 (3)	total: 49.3ms	remaining: 1.18s
4:	learn: 5997.9656398	test: 6024.1467882	best: 6024.1467882 (4)	total: 63.1ms	remaining: 1.2s
5:	learn: 5866.3646941	test: 5892.4168626	best: 5892.4168626 (5)	total: 79ms	remaining: 1.24s
6:	learn: 5739.1651956	test: 5765.5593263	best: 5765.5593263 (6)	total: 94ms	remaining: 1.25s
7:	learn: 5616.9271735	test: 5643.3950019	best: 5643.3950019 (7)	total: 106ms	remaining: 1.22s
8:	learn: 5499.8535161	test: 5526.6693614	best: 5526.6693614 (8)	total: 117ms	remaining: 1.19s
9:	learn: 5385.9380721	test: 5412.8583289	best: 5412.8583289 (9)	total: 128ms	remaining: 1.15s
10:	learn: 5275.4277063	test: 5302.5002992	best: 

Проверка данных на тестовой выборке подтверждает ранее сделанные выводы. RMSE составляет 1979 и с запасом укладывается в минимальное требование в 2500. Модель рекомендована к применению!